# Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

In [1]:
# from IPython.display import Video
# Video("Project 17 - Custom Callbacks.webm",  embed=True)

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Version:  2.9.1


# Examples of Built-in Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [3]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised = True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling horses_or_humans-train.tfrecord...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling horses_or_humans-test.tfrecord...:   0%|          | 0/256 [00:00<?, ? examples/s]

Dataset horses_or_humans downloaded and prepared to C:\Users\pooya\tensorflow_datasets\horses_or_humans\3.0.0. Subsequent calls will reuse this data.


In [4]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [5]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [6]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [7]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [8]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [9]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [10]:
!rm -rf logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 10s 244ms/step - loss: 0.6809 - accuracy: 0.5449 - val_loss: 0.6542 - val_accuracy: 0.5561
Epoch 2/10
26/26 [==============================] - 4s 140ms/step - loss: 0.6249 - accuracy: 0.6617 - val_loss: 0.5857 - val_accuracy: 0.8244
Epoch 3/10
26/26 [==============================] - 4s 140ms/step - loss: 0.5713 - accuracy: 0.7398 - val_loss: 0.5038 - val_accuracy: 0.8146
Epoch 4/10
26/26 [==============================] - 4s 140ms/step - loss: 0.4982 - accuracy: 0.7944 - val_loss: 0.4460 - val_accuracy: 0.8341
Epoch 5/10
26/26 [==============================] - 4s 142ms/step - loss: 0.3947 - accuracy: 0.8406 - val_loss: 0.3480 - val_accuracy: 0.8829
Epoch 6/10
26/26 [==============================] - 4s 142ms/step - loss: 0.3752 - accuracy: 0.8650 - val_loss: 0.2817 - val_accuracy: 0.9073
Epoch 7/10
26/26 [==============================] - 4s 140ms/step - loss: 0.3238 - accuracy: 0.8938 - val_loss: 0.2430 - val_accuracy: 0.9171
Epoch

In [12]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 4s - loss: 0.6711 - accuracy: 0.5827 - val_loss: 0.6493 - val_accuracy: 0.7463

Epoch 00001: saving model to weights.01-0.65.h5
Epoch 2/5
26/26 - 4s - loss: 0.6260 - accuracy: 0.6898 - val_loss: 0.6465 - val_accuracy: 0.5805

Epoch 00002: saving model to weights.02-0.65.h5
Epoch 3/5
26/26 - 4s - loss: 0.5968 - accuracy: 0.6898 - val_loss: 0.6506 - val_accuracy: 0.5463

Epoch 00003: saving model to weights.03-0.65.h5
Epoch 4/5
26/26 - 4s - loss: 0.5876 - accuracy: 0.7032 - val_loss: 0.5219 - val_accuracy: 0.8293

Epoch 00004: saving model to weights.04-0.52.h5
Epoch 5/5
26/26 - 4s - loss: 0.5070 - accuracy: 0.7822 - val_loss: 0.4972 - val_accuracy: 0.7951

Epoch 00005: saving model to weights.05-0.50.h5


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

26/26 - 4s - loss: 0.6712 - accuracy: 0.5815 - val_loss: 0.7149 - val_accuracy: 0.4488

Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 4s 146ms/step - loss: 0.6795 - accuracy: 0.5841 - val_loss: 0.6623 - val_accuracy: 0.5512
Epoch 2/5
26/26 [==============================] - 4s 141ms/step - loss: 0.6344 - accuracy: 0.6609 - val_loss: 0.6138 - val_accuracy: 0.7024
Epoch 3/5
26/26 [==============================] - 4s 140ms/step - loss: 0.5818 - accuracy: 0.7544 - val_loss: 0.5945 - val_accuracy: 0.6488
Epoch 4/5
26/26 [==============================] - 4s 140ms/step - loss: 0.5184 - accuracy: 0.7509 - val_loss: 0.5128 - val_accuracy: 0.7951
Epoch 5/5
26/26 [==============================] - 4s 140ms/step - loss: 0.4659 - accuracy: 0.7780 - val_loss: 0.4193 - val_accuracy: 0.7951


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.618005,0.668965,0.551220,0.662288
1,1,0.681265,0.627584,0.702439,0.613802
2,2,0.755474,0.563054,0.648780,0.594503
3,3,0.778589,0.505214,0.795122,0.512806
4,4,0.796837,0.451036,0.795122,0.419333


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
26/26 [==============================] - 4s 155ms/step - loss: 0.6938 - accuracy: 0.5521 - val_loss: 0.7139 - val_accuracy: 0.4341
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
26/26 [==============================] - 4s 140ms/step - loss: 0.6471 - accuracy: 0.5986 - val_loss: 0.6318 - val_accuracy: 0.7610
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
26/26 [==============================] - 4s 140ms/step - loss: 0.6125 - accuracy: 0.7300 - val_loss: 0.6201 - val_accuracy: 0.7756
Epoch 4/5

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
26/26 [==============================] - 4s 140ms/step - loss: 0.6036 - accuracy: 0.7695 - val_loss: 0.6103 - val_accuracy: 0.8098
Epoch 5/5

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
26/26 [==============================] - 4s 140ms/step - loss: 0.5981 -

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1, # factor by which the learning rate will be reduced. new_lr = lr * factor. 
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

In [ ]:
%tensorboard --logdir log_dir

# Keras custom callbacks

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import datetime
import io

from PIL import Image
from IPython.display import Image as IPyImage
import imageio

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.4.0


In [2]:
# Define the Keras model to add callbacks to
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1, activation = 'linear', input_dim = 784))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='mean_squared_error', metrics=['mae'])
    return model

In [3]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

Now, define a simple custom callback to track the start and end time of every batch of data. During those calls, it prints the index of the current batch.

In [4]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_train_batch_begin(self, batch, logs=None):
        print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

    def on_train_batch_end(self, batch, logs=None):
        print('Training: batch {} ends at {}'.format(batch, datetime.datetime.now().time()))

Providing a callback to model methods such as `tf.keras.Model.fit()` ensures the methods are called at those stages:

In [8]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=128,
          epochs=1,
          steps_per_epoch=5, # when we set this, wetrain the model on steps_per_epoch num of batches not all training set 
          verbose=0,
          callbacks=[MyCustomCallback()])

Training: batch 0 begins at 23:57:47.495810
Training: batch 0 ends at 23:57:47.843444
Training: batch 1 begins at 23:57:47.843444
Training: batch 1 ends at 23:57:47.843444
Training: batch 2 begins at 23:57:47.843444
Training: batch 2 ends at 23:57:47.843444
Training: batch 3 begins at 23:57:47.843444
Training: batch 3 ends at 23:57:47.843444
Training: batch 4 begins at 23:57:47.843444
Training: batch 4 ends at 23:57:47.859065


## An overview of callback methods


### Common methods for training/testing/predicting
For training, testing, and predicting, following methods are provided to be overridden.
#### `on_(train|test|predict)_begin(self, logs=None)`
Called at the beginning of `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_end(self, logs=None)`
Called at the end of `fit`/`evaluate`/`predict`.
#### `on_(train|test|predict)_batch_begin(self, batch, logs=None)`
Called right before processing a batch during training/testing/predicting. Within this method, `logs` is a dict with `batch` and `size` available keys, representing the current batch number and the size of the batch.
#### `on_(train|test|predict)_batch_end(self, batch, logs=None)`
Called at the end of training/testing/predicting a batch. Within this method, `logs` is a dict containing the stateful metrics result.

### Training specific methods
In addition, for training, following are provided.
#### `on_epoch_begin(self, epoch, logs=None)`
Called at the beginning of an epoch during training.
#### `on_epoch_end(self, epoch, logs=None)`
Called at the end of an epoch during training.


### Usage of `logs` dict
The `logs` dict contains the loss value, and all the metrics at the end of a batch or epoch. Example includes the loss and mean absolute error.

In [11]:
callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch,logs: 
    print("Epoch: {}, Val/Train loss ratio: {:.2f}".format(epoch, logs["val_loss"] / logs["loss"]))
)

model = get_model()
_ = model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=64,
          epochs=3,
          verbose=0,
          callbacks=[callback])

Epoch: 0, Val/Train loss ratio: 1.85
Epoch: 1, Val/Train loss ratio: 0.89
Epoch: 2, Val/Train loss ratio: 0.41


In [12]:
class DetectOverfittingCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold=0.7):
        super(DetectOverfittingCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        ratio = logs["val_loss"] / logs["loss"]
        print("Epoch: {}, Val/Train loss ratio: {:.2f}".format(epoch, ratio))

        if ratio > self.threshold:
            print("Stopping training...")
            self.model.stop_training = True

model = get_model()
_ = model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=64,
              epochs=3,
              verbose=0,
              callbacks=[DetectOverfittingCallback()])

Epoch: 0, Val/Train loss ratio: 0.44
Epoch: 1, Val/Train loss ratio: 0.40
Epoch: 2, Val/Train loss ratio: 1.57
Stopping training...


Similarly, one can provide callbacks in `evaluate()` calls.

## Custom callback to Visualize predictions

In [13]:
# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

In [ ]:
# Visualization utilities
plt.rc('font', size=20)
plt.rc('figure', figsize=(15, 3))

def display_digits(inputs, outputs, ground_truth, epoch, n=10):
    plt.clf()

    plt.yticks([])
    plt.grid(None)
    inputs = np.reshape(inputs, [n, 28, 28])
    inputs = np.swapaxes(inputs, 0, 1)
    inputs = np.reshape(inputs, [28, 28*n])
    plt.imshow(inputs)
    plt.xticks([28*x+14 for x in range(n)], outputs)
    for i,t in enumerate(plt.gca().xaxis.get_ticklabels()):
        if outputs[i] == ground_truth[i]: 
            t.set_color('green') 
        else: 
            t.set_color('red')
    plt.grid(None)

In [ ]:
GIF_PATH = './animation.gif'

In [ ]:
class VisCallback(tf.keras.callbacks.Callback):
    def __init__(self, inputs, ground_truth, display_freq=10, n_samples=10):
        self.inputs = inputs
        self.ground_truth = ground_truth
        self.images = []
        self.display_freq = display_freq
        self.n_samples = n_samples

    def on_epoch_end(self, epoch, logs=None):
        # Randomly sample data
        indexes = np.random.choice(len(self.inputs), size=self.n_samples)
        X_test, y_test = self.inputs[indexes], self.ground_truth[indexes]
        predictions = np.argmax(self.model.predict(X_test), axis=1)

        # Plot the digits
        display_digits(X_test, predictions, y_test, epoch, n=self.display_freq)

        # Save the figure
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = Image.open(buf)
        self.images.append(np.array(image))

        # Display the digits every 'display_freq' number of epochs
        if epoch % self.display_freq == 0:
            plt.show()

    def on_train_end(self, logs=None):
        imageio.mimsave(GIF_PATH, self.images, fps=1)

In [ ]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='linear', input_dim=784))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = get_model()
model.fit(x_train, y_train,
          batch_size=64,
          epochs=20,
          verbose=0,
          callbacks=[VisCallback(x_test, y_test)])